# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Note: pandas and numpy are already imported as pd and np
# The following tables are loaded as pandas DataFrames with the same names: fct_guest_spending
# Please print your final result or dataframe
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'])

july_df = fct_guest_spending[
    (fct_guest_spending['visit_date'].dt.month == 7) &
    (fct_guest_spending['visit_date'].dt.year == 2024)
]

guest_park_spend = july_df.groupby(['guest_id', 'park_experience_type'])['amount_spent'].sum().reset_index()

guest_park_visits = july_df.groupby(['guest_id', 'park_experience_type'])['visit_date'].count().reset_index()
guest_park_visits.rename(columns={'visit_date': 'visit_count'}, inplace=True)

merged = pd.merge(guest_park_spend, guest_park_visits, on=['guest_id', 'park_experience_type'])

merged['avg_spend_per_visit'] = merged['amount_spent'] / merged['visit_count']

final_result = merged.groupby('park_experience_type')['avg_spend_per_visit'].mean().reset_index()
all_types = fct_guest_spending['park_experience_type'].unique()
final_result = final_result.set_index('park_experience_type').reindex(all_types, fill_value=0.0).reset_index()

print(final_result)

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date']).dt.normalize()
aug_df = fct_guest_spending[
  (fct_guest_spending['visit_date'].dt.month == 8) &
  (fct_guest_spending['visit_date'].dt.year == 2024)]
guest_park_visits = aug_df.groupby('guest_id')['visit_date'].count().reset_index()
guest_park_visits = guest_park_visits.rename(columns={'visit_date':'visit_count'})
repeat_guests = guest_park_visits[guest_park_visits['visit_count'] > 1 ]
aug_repeat_df = aug_df[aug_df['guest_id'].isin(repeat_guests['guest_id'])]
aug_repeat_df = aug_repeat_df.sort_values(by=['guest_id', 'visit_date'])
first_last = aug_repeat_df.groupby('guest_id').agg(
  first_visit =('visit_date', 'first'), last_visit=('visit_date', 'last')).reset_index()
first_spend = aug_repeat_df.merge(first_last[['guest_id', 'first_visit']],
                                 left_on=['guest_id', 'visit_date'],
                                 right_on=['guest_id', 'first_visit'])[['guest_id', 'amount_spent']]
first_spend.rename(columns={'amount_spent':'first_spending'}, inplace=True)
last_spend = aug_repeat_df.merge(first_last[['guest_id', 'last_visit']],
                                 left_on=['guest_id', 'visit_date'],
                                 right_on=['guest_id', 'last_visit'])[['guest_id', 'amount_spent']]
last_spend.rename(columns={'amount_spent':'last_spending'}, inplace=True)

merged_spending = pd.merge(first_spend, last_spend, on='guest_id')
merged_spending['spending_diff'] = merged_spending['last_spending'] - merged_spending['first_spending']
print(merged_spending[['guest_id', 'first_spending', 'last_spending', 'spending_diff']])

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date']).dt.normalize()
sept_df = fct_guest_spending[
  (fct_guest_spending['visit_date'].dt.month == 9) &
  (fct_guest_spending['visit_date'].dt.year == 2024)]
sept_spent = sept_df.groupby('guest_id')['amount_spent'].sum().reset_index()
sept_spent = sept_spent.rename(columns={'amount_spent':'total_spent_sept'})
sept_spent = sept_spent[sept_spent['total_spent_sept'] > 0]
conditions = [
  (sept_spent['total_spent_sept'] < 50),
  (sept_spent['total_spent_sept'] >= 50) & (sept_spent['total_spent_sept'] < 100),
  (sept_spent['total_spent_sept'] >= 100)]

choices = ['Low', 'Medium', 'High']

sept_spent['spending_segment'] = np.select(conditions, choices, default='Unknown')

print(sept_spent[['guest_id', 'total_spent_sept', 'spending_segment']])

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)